# Operations continued
We have been using a visual editor to create quantum circuits throughout the course. Now it's your turn to create some circuits on your own using qrisp, the next generation  quantum programming framework!

This notebook will help you deepen your understanding of quantum operations and universal gate sets introduced in the lecture.

You will:
- Review single- and two-qubit gates.
- Learn about universal gate sets and circuit transpilation.
- Specify the physical qubits using **Qrisp**.
- Transpile multi-controlled gates into a universal two-qubit gate set.


In order to use qrisp together with IQM devices, please make sure to install the latest version of qrisp. 

In [ ]:
%pip install "iqm-client[qiskit] >= 32.1.1, < 33.0"
%pip install "qrisp[iqm]"

## Universal Gate Sets

A **universal gate set** allows any quantum computation to be approximated to arbitrary accuracy.

Common examples:
- {H, T, CNOT}
- {Rz, Rx, CZ}
- {CZ, R}  (used in this notebook)

The process of **transpilation** converts a general quantum circuit into one that uses only gates from the chosen universal set.

**Task: Use the `qc.transpile(basis_gates=["cz", "r"])` method to transpile the circuit and compare it to the untranspiled version.**


In [ ]:
# Import the Qrisp quantum library
from qrisp import *

# Define two qubits as a quantum variable
q = QuantumVariable(2)

# Apply a Hadamard to the first qubit
h(q[0])

# Apply a CNOT (control: q[0], target: q[1])
cx(q[0], q[1])

# Visualize and compile
qc = q.qs.compile()
print(qc)

print("TODO: print the transpiled circuit diagram here")


## Multi-Controlled Gates and Decomposition

The **multi-controlled X gate (MCX)** generalizes the CNOT gate.  
For example, `mcx([0,1,2], 3)` flips qubit 3 if all control qubits (0,1,2) are in state `1`.

However, **quantum hardware** typically only supports *two-qubit gates*, so multi-controlled gates must be **decomposed** into equivalent two-qubit operations.

**Task: Create a circuit that uses the MCX gate and transpile it to see how it is decomposed into two-qubit gates.**


In [ ]:
# Define 4 qubits
qv = QuantumVariable(4)

# Apply a multi-controlled X (at least three controls -> one target)

# Compile and transpile

# Transpile to two-qubit universal gate set


**Task: MCX Decomposition**

1. Observe the above transpiled version of the `mcx` gate (tip: use `__dir__()` on the object return by the `transpile` method).
2. How many **two-qubit** gates does it require?
3. Compare the number of required cz gates to the number of required iswap gates when considering the universal gate set that includes iSWAP as the two-qubit gate {iSWAP, R}.
4. For the quick ones: Try building your own 3-control gate manually using `cx` and `cz` gates only. 

When running on hardware, this transpilation is automatically handled for us, however, it makes sense to understand what is happening "under the hood".

## Improving outcomes by selecting physical qubits

A typical quantum processor consists of multiple qubits with varying coherence times and gate fidelities. By selecting the best physical qubits for your circuit, you can improve the overall performance and reliability of your quantum computations.

This can be done in any framework. For qrisp, the following is a way to do it:

**Task: But, oops, the coupling map is incomplete, check IQM Resonance's coupling map and add at least one edge to give the transpiler enough freedom to map the circuit.**

In [ ]:
from qrisp.interface import IQMBackend

import qiskit
# Define a custom transpilation procedure (default will be default 
# qiskit transpiler to match connectivity and gate set).
# Should take a Qiskit circuit and return a Qiskit circuit


qb_index = lambda q: int(q.strip().upper().replace("QB", "")) - 1 if q.strip().upper().startswith("QB") else None


def custom_transpilation(qc : qiskit.QuantumCircuit):
    # TODO: Check and change only the next line
    couplings = [["QB15","QB16"],["QB16","QB24"],["QB23","QB24"],["QB23","QB15"]] 
    reduced_coupling_map = [[qb_index(q) for q in row] for row in couplings]
    qc = qiskit.transpile(qc, basis_gates = ["cz", "r", "measure", "reset"], coupling_map=reduced_coupling_map, optimization_level=3)
        
    return qc
    
# Create a backend object
quantum_computer = IQMBackend(api_token = input("Enter your IQM Resonance API token: "), 
                          device_instance = "emerald",
                          transpiler = custom_transpilation)


qv = QuantumVariable(5)

x(qv[0])
cx(qv[0], qv[1])
cx(qv[1], qv[2])
cx(qv[2], qv[3])
cx(qv[3], qv[4])


meas_res = qv.get_measurement(backend = quantum_computer)

**Task: Then, try to find qubits with low readout errors and good two-qubit gate fidelities for your circuit.**